# Logan Clark - College To NFL Trends (RB)

Predicting a players NFL Fantasy Football success by using College Football stats along with Random Forest Regression, and Neural Network Regression. 

**Data**

Evaluation Methodology: The current value metric I am using for each player is

Value = (AVG EP Season 1-3 + AVG PPG Season 1-3)/2 + (#Top-12 Finishes * 2)

My logic behind this is this first part averages a players Expected Points and Actual Points. And then I need the second part because I only have Season 1-3 stats for EP and PPG and how long a player performs at a high level is relevant to Dynasty Purposes.





Source: Pahowdy's Database

https://docs.google.com/spreadsheets/d/1D1D9rXo3BXX867qBju77VjZ9YIoAp-nXYFqSGDQgajU/edit#gid=224755041











In [13]:
!pip install tensorflow-addons

import math
import time

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa

import torch

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

tf.keras.backend.set_floatx('float64')

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 

from helper_functions import create_tensorboard_callback, plot_loss_curves

--2021-10-17 23:13:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-17 23:13:35 (49.4 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [14]:
def eval_model(model, X_test, y_test, hist):
    print("eval", model.evaluate(X_test, y_test))
    print("predict", model.predict(X_test[:10]))
    pd.DataFrame(hist.history).plot()
    plt.ylabel("loss")
    plt.xlabel("epochs")

metrics = [tf.metrics.mae, tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))]
    

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Data Prep

In [16]:
data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_rb_training.csv')
rookie_data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_rb_rookies.csv')

features = (data.columns)[4:98]

print(data.head())
print(rookie_data.head())

data = data.to_numpy()
rookie_data = rookie_data.to_numpy()
rookie_names = rookie_data[:,0]

data = np.delete(data, 0, axis=0)



                  Name  ... Year 1-3 Score + RB1
0                    0  ...              109.000
1  LaDainian Tomlinson  ...               38.865
2      Adrian Peterson  ...               27.075
3        Malcolm Perry  ...                2.615
4       Anthony Thomas  ...               13.165

[5 rows x 110 columns]
               Name              cfb_id    pfr_id  ... Burst Score   HaSS   WaSS
0      Najee Harris      najee-harris-1  HarrNa00  ...      118.56  88.24  99.04
1    Travis Etienne    travis-etienne-1  EtieTr00  ...      120.95  88.24  99.04
2  Javonte Williams  javonte-williams-1  WillJa10  ...      122.70  88.24  99.04
3       Trey Sermon       trey-sermon-1  SermTr00  ...      125.40  88.24  99.04
4    Michael Carter    michael-carter-5  CartMi02  ...      117.30  83.31  95.10

[5 rows x 97 columns]


In [17]:
def encode_cfb_data(input_data):
  ConferenceDict = {'ACC':0, 'ACC (Atlantic)':1, 'ACC (Coastal)':2, 'American':3, 'American (East)':4, 'American (West)':5,
                  'Big 12':6, 'Big 12 (North)':7, 'Big 12 (South)':8, 'Big East':9, 'Big Ten':10, 'Big Ten (East)':11,
                  'Big Ten (Leaders)':12, 'Big Ten (Legends)':13, 'Big Ten (West)':14, 'CUSA': 15, 'CUSA (East)':16,
                  'CUSA (West)':17, 'Ind':18, 'MAC (East)':19, 'MAC (West)':20, 'MWC':21, 'MWC (Mountain)':22,
                  'MWC (West)':23, 'Pac-10':24, 'Pac-12 (North)':25, 'Pac-12 (South)':26, 'SEC (East)':27,
                  'SEC (West)':28, 'Sun Belt':29, 'Sun Belt (East)':30, 'Sun Belt (West)':31, 'WAC':31}

  DraftRoundDict = {'UDFA':9}

  DraftPositionDict = {'UDFA':260}

  LastYearDesignationDict = {'SR':0, 'JR':1, 'SO':2, 'FR':3}

  for player in input_data:
    player[4] = ConferenceDict[(player[4].replace("\xa0", " "))]
    if not (player[5].isdigit()):
      player[5] = DraftRoundDict[player[5]]
    if not (player[6].isdigit()):
      player[6] = DraftPositionDict[player[6]]
    player[7] = LastYearDesignationDict[player[7]]

  return input_data

In [18]:
# Data Encoding
data = encode_cfb_data(data)
rookie_data = encode_cfb_data(rookie_data)

In [19]:
# Split data into X and y

X = list()
y = list()

X_rookie = list()

for player in data:
  X.append(player[4:97])
  y.append(player[109])

for player in rookie_data:
  X_rookie.append(player[4:97])

X = np.array(X).astype(float) 
y = np.array(y).astype(float)

X_rookie = np.array(X_rookie).astype(float)

In [20]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Baseline Random Forest Model

In [21]:
rfReg = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
rf_r_squared = rfReg.score(X_test, y_test)
rf_r_squared

0.5025486906068686

In [22]:
#List features by importance

feature_importance = rfReg.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.18420223658868437, 'DP')
(0.09235786775066279, 'Post Draft Percentile')
(0.08656224258855426, 'Age Draft Hit Rate AVG')
(0.06873110918341867, 'Post Draft Model')
(0.022369450309152952, 'YPTch')
(0.01769395587625197, 'Best.9')
(0.016649427418365545, 'Production Score')
(0.014300670469235104, 'First.2')
(0.0139946746483325, 'AVG YD/G')
(0.013628540803351342, 'Last.9')
(0.013019479287922926, 'Last.8')
(0.011446377117850626, 'Rush College Dominator Rating')
(0.011083835765586483, 'First.11')
(0.01099548120156407, 'vert')
(0.010761884180028823, 'Best.10')
(0.010164282208527024, '40')
(0.009778667049935987, 'REC/g')
(0.009761116465797278, 'First.5')
(0.009709102747342225, 'Burst Score')
(0.009657993332051123, 'AVG.8')
(0.009384464888582422, 'AVG.11')
(0.009324038697192628, 'Best.8')
(0.009137878087786388, 'Conference')
(0.008886428535112569, 'First.9')
(0.00880532187612179, 'shuttle')
(0.008781326132819612, 'First.10')
(0.008458991000447029, 'All Yards')
(0.008348755886323073, 'First.8')


# Baseline Neural Network

In [24]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Pre-processing layer (normalization)
data_normalization = Sequential([
                                 preprocessing.Normalization()
], name="data_normalization")

# Setup input shape
input_shape = 93

# Assemble model
inputs = layers.Input(shape=input_shape, name="input_layer")
x = data_normalization(inputs)
x = layers.Dense(100)(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(10)(x)
x = layers.LeakyReLU()(x)
outputs = layers.Dense(1, name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=metrics)

history_0 = model_0.fit(X_train, y_train, epochs=50, callbacks=[create_tensorboard_callback(dir_name="rb_trends",
                                                                                            experiment_name="model_0")])

Saving TensorBoard log files to: rb_trends/model_0/20211017-231606
Epoch 1/50
13/13 [==============================] - 4s 27ms/step - loss: 59.6317 - mean_absolute_error: 59.6317 - r_square: -253.9348
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 17.1583 - mean_absolute_error: 17.1583 - r_square: -13.4620
Epoch 3/50
13/13 [==============================] - 0s 5ms/step - loss: 12.3803 - mean_absolute_error: 12.3803 - r_square: -6.2163
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 9.1544 - mean_absolute_error: 9.1544 - r_square: -3.2575
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 8.4521 - mean_absolute_error: 8.4521 - r_square: -2.6138
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 6.2495 - mean_absolute_error: 6.2495 - r_square: -1.0666
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 5.3820 - mean_absolute_error: 5.3820 - r_square: -0.5311
Epoch 8/50
13/13 [==

In [25]:
model_0.evaluate(X_test, y_test)

4/4 [==============================] - 0s 5ms/step - loss: 5.2164 - mean_absolute_error: 5.2164 - r_square: -0.1863


[5.216370195445448, 5.216370011299144, -0.1862565279006958]

In [26]:
model_1 = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1.fit(X_train, y_train, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 6ms/step - loss: 23.4377 - mean_absolute_error: 23.4377 - r_square: -22.9166
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: 5.1484 - mean_absolute_error: 5.1484 - r_square: -0.4893
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 4.2255 - mean_absolute_error: 4.2255 - r_square: -0.1042
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: 3.6417 - mean_absolute_error: 3.6417 - r_square: 0.1201
Epoch 5/50
13/13 [==============================] - 0s 5ms/step - loss: 3.9073 - mean_absolute_error: 3.9073 - r_square: -0.0103
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 3.8777 - mean_absolute_error: 3.8777 - r_square: 0.0884
Epoch 7/50
13/13 [==============================] - 0s 6ms/step - loss: 3.4209 - mean_absolute_error: 3.4209 - r_square: 0.1954
Epoch 8/50
13/13 [==============================] - 0s 5ms/step - loss: 3.5207 - mean_absolute_er

In [27]:
model_1.evaluate(X_test, y_test)

4/4 [==============================] - 0s 5ms/step - loss: 3.5294 - mean_absolute_error: 3.5294 - r_square: 0.4890


[3.529383121150555, 3.5293830413365486, 0.4889867901802063]

In [28]:
def normalized_predictions_with_names(predictions):
  pred_sum = sum(predictions)

  normalized_preds = [(float(i)/pred_sum)*1000 for i in predictions]

  names_preds = dict()

  for i in range(len(normalized_preds)):
      if type(normalized_preds[i]) is np.ndarray:
        names_preds[rookie_names[i]] = normalized_preds[i][0]
      else:
        names_preds[rookie_names[i]] = normalized_preds[i]   
          
  sorted_rookies = sorted(names_preds.items(), key=lambda x: x[1], reverse=True)

  # for i in sorted_rookies:
  #     print(i)
  return sorted_rookies

In [29]:
rf_preds = rfReg.predict(X_rookie)
rf_preds_norm = normalized_predictions_with_names(rf_preds)

for i in rf_preds_norm:
  print(i)

('Najee Harris', 50.75983892660296)
('Travis Etienne', 47.7113932210464)
('Javonte Williams', 39.57168062808425)
('Trey Sermon', 38.131158626209675)
('Elijah Collins', 26.633524020929887)
('Michael Carter', 26.4097435076329)
('Chuba Hubbard', 26.268015849211476)
('Pooka Williams', 24.5962539835967)
('Jermar Jefferson', 23.118261756239875)
('Spencer Brown', 21.186810411240927)
('Rakeem Boyd', 21.015418824312686)
('CJ Verdell', 20.791464837749576)
('Isaiah Bowser', 20.61955283102298)
('Kenny Gainwell', 20.619205884490736)
('Josh Johnson', 20.109367955359073)
('Kennedy Brooks', 19.87240347383685)
('Elijah Mitchell', 19.825912638516225)
('Malik Davis', 19.61392830731551)
('Gerrid Doaks', 19.53239587223833)
('Rhamondre Stevenson', 19.373667833736967)
('JaTarvious Whitlow', 19.260216317693377)
('Jafar Armstrong', 19.17955124894679)
('B.J. Emmons', 18.6246102706235)
('Stevie Scott', 18.52069978421661)
('Jaret Patterson', 18.470739483573553)
('Jerome Ford', 18.084934939718913)
('Journey Brown'

In [30]:
model_1_preds = model_1.predict(X_rookie)
model_1_preds_norm = normalized_predictions_with_names(model_1.predict(X_rookie))

for i in model_1_preds_norm:
  print(i)

('Travis Etienne', 99.96607083078258)
('Najee Harris', 79.52815333054484)
('Javonte Williams', 65.56410841353893)
('Trey Sermon', 60.33966319396718)
('Chuba Hubbard', 58.88118344688494)
('Michael Carter', 48.82872278050449)
('Kenny Gainwell', 34.91481293745662)
('Rhamondre Stevenson', 28.109651445995446)
('Elijah Mitchell', 25.432042134246615)
('Jermar Jefferson', 22.935324191574896)
('Kene Nwangwu', 22.13597336059029)
('Jaret Patterson', 21.392507446573063)
('CJ Verdell', 20.99452340278856)
('Pooka Williams', 18.469599819368412)
('Kennedy Brooks', 15.253064702814967)
('Spencer Brown', 15.233377338030682)
('Chase Hayden', 15.023391114901766)
('Kylin Hill', 14.14074732492406)
('Jerome Ford', 14.04149071949768)
('Jafar Armstrong', 13.770222907994123)
('Khalil Herbert', 13.277750078121647)
('Brenden Knox', 12.208917667750633)
('B.J. Emmons', 12.1011431076666)
('Alex Fontenot', 12.086633157362543)
('Gerrid Doaks', 11.875841293301455)
('Stevie Scott', 11.833709099394166)
('Elijah Collins', 

# Model Comparison

* Similar R^2 values
* Random forest seems incremental with it's values while Model_2 is the opposite
* Ex: 0-->1 in model_2 drops 12 value points and 5-->6 drops 11 value points, largest drop in rf is 2-->3 with 4 value points.

# Training with Data that does not involve Draft Position to search for "Sleepers"

Theory: Players that's value increases the most when the data does not involve draft position are talented players who were not drafted high meaning they would have a higher % chance to breakout than the average player at their draft position.

In [31]:
# Split data into X and y

X_sleeper = list()
y_sleeper = list()

X_sleeper_rookie = list()

for player in data:
  temp1 = player[7:14]
  temp2 = player[16:97]
  X_sleeper.append(list(temp1)+list(temp2))
  # X_sleeper.append(player[7:97])
  y_sleeper.append(player[109])

for player in rookie_data:
  temp1 = player[7:14]
  temp2 = player[16:97]
  X_sleeper_rookie.append(list(temp1)+list(temp2))
  #X_sleeper_rookie.append(player[7:98])

X_sleeper = np.array(X_sleeper).astype(float) 
y_sleeper = np.array(y_sleeper).astype(float)

X_sleeper_rookie = np.array(X_sleeper_rookie).astype(float)

In [32]:
print(X_sleeper[0])

[0.0000e+00 2.2000e+01 2.5000e-02 1.4300e-01 8.1860e-01 1.4500e+01
 1.0000e+00 1.0000e+00 0.0000e+00 3.6900e+02 2.1580e+03 1.9620e+02
 1.6510e+02 5.8000e+00 1.1000e+01 0.0000e+00 0.0000e+00 3.6600e-01
 3.1000e+00 1.2000e+00 2.1000e+00 1.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 1.0000e+00 3.1000e+00 3.1000e+00 3.1000e+00
 3.1000e+00 2.1000e+00 2.1000e+00 2.1000e+00 2.1000e+00 3.1000e+00
 3.1000e+00 3.1000e+00 3.1000e+00 3.1000e+00 3.1000e+00 3.1000e+00
 3.1000e+00 1.2000e+00 1.2000e+00 1.2000e+00 1.2000e+00 2.1000e+00
 2.1000e+00 2.1000e+00 2.1000e+00 7.3200e-01 3.6600e-01 3.6600e-01
 3.6600e-01 3.6600e-01 6.0000e-02 6.0000e-02 6.0000e-02 0.0000e+00
 8.5000e-01 8.5000e-01 8.5000e-01 0.0000e+00 6.0800e-01 3.4780e+02
 3.4780e+02 3.4780e+02 0.0000e+00 3.1620e+01 3.1620e+01 3.1620e+01
 0.0000e+00 1.0000e-03 2.6000e+00 1.6200e+01 0.0000e+00 2.2100e+02
 7.0000e+01 1.8000e+01 4.4600e+00 6.8400e+00 4.2100e+00 4.0500e+01
 1.2400e+02 1.3085e+02 1.0115e+02 1.1171e+02]


In [33]:
# Split data into train and test
X_train_sleeper, X_test_sleeper, y_train_sleeper, y_test_sleeper = train_test_split(X_sleeper, y_sleeper, test_size=0.2, random_state=42)

X_train_sleeper = tf.convert_to_tensor(X_train_sleeper)
X_test_sleeper = tf.convert_to_tensor(X_test_sleeper)
y_train_sleeper = tf.convert_to_tensor(y_train_sleeper)
y_test_sleeper = tf.convert_to_tensor(y_test_sleeper)

In [34]:
#Train Random Forest
rf_sleeper = RandomForestRegressor(n_estimators=100).fit(X_train_sleeper, y_train_sleeper)
rf_r_squared_sleeper = rf_sleeper.score(X_test_sleeper, y_test_sleeper)
rf_r_squared_sleeper

0.4585219406924303

In [45]:
#List features by importance

temp1 = features[3:10]
temp2 = features[12:]

sleeper_features = list(temp1) + list(temp2)



feature_importance = rf_sleeper.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], sleeper_features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.3804895806826632, 'Age Draft Hit Rate AVG')
(0.023979371109997188, 'Production Score')
(0.023003699753679784, 'Last.9')
(0.019005242092755472, 'First.11')
(0.017531393459083562, 'Best.9')
(0.01599408033280075, 'YPTch')
(0.013393693089270757, 'First.1')
(0.012437888263983959, 'First.2')
(0.012280661328116949, 'Conference AVG')
(0.012223799966133875, 'Best')
(0.0121844078080475, 'AVG YD/G')
(0.011836405982089883, 'Conference Drafted Rate')
(0.011555161529744534, 'First.5')
(0.011553747433007215, '40')
(0.01108591317946739, 'Rush College Dominator Rating')
(0.010710923019567099, 'Last.8')
(0.010613358672365795, 'bench')
(0.010284290237548325, 'First.7')
(0.009899343706671507, 'Touches')
(0.009889701810337126, 'First.8')
(0.009767819152486304, 'Last')
(0.009756898488871858, 'vert')
(0.009655639442223456, 'AVG.8')
(0.00954367930331363, 'Best.10')
(0.009405296265422407, 'YD/G')
(0.008894123545500916, 'First')
(0.008846785049641674, 'Last.10')
(0.008499629634044166, 'AVG.9')
(0.00847590540

In [36]:
# Retrain model_1 on new data
model_1_sleeper = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1_sleeper.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1_sleeper.fit(X_train_sleeper, y_train_sleeper, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 6ms/step - loss: 24.4714 - mean_absolute_error: 24.4714 - r_square: -34.4255
Epoch 2/50
13/13 [==============================] - 0s 5ms/step - loss: 4.3404 - mean_absolute_error: 4.3404 - r_square: -0.2271
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 4.2701 - mean_absolute_error: 4.2701 - r_square: -0.1264
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: 4.3809 - mean_absolute_error: 4.3809 - r_square: -0.1476
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 4.0434 - mean_absolute_error: 4.0434 - r_square: -0.1111
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 3.8321 - mean_absolute_error: 3.8321 - r_square: -0.0081
Epoch 7/50
13/13 [==============================] - 0s 5ms/step - loss: 4.0406 - mean_absolute_error: 4.0406 - r_square: -0.0419
Epoch 8/50
13/13 [==============================] - 0s 6ms/step - loss: 3.9152 - mean_absolute

In [37]:
model_1_sleeper.evaluate(X_test_sleeper, y_test_sleeper)

4/4 [==============================] - 0s 8ms/step - loss: 4.0290 - mean_absolute_error: 4.0290 - r_square: 0.1502


[4.028963674413095, 4.028963478342029, 0.1501789093017578]

In [38]:
rf_preds_sleeper = rf_sleeper.predict(X_sleeper_rookie)
rf_preds_norm_sleeper = normalized_predictions_with_names(rf_sleeper.predict(X_sleeper_rookie))

for i in rf_preds_norm_sleeper:
  print(i)


('Travis Etienne', 46.66382786161645)
('Javonte Williams', 46.13805560794644)
('Trey Sermon', 46.12111177555272)
('Najee Harris', 41.573318701164546)
('Jermar Jefferson', 29.35818474391024)
('Chuba Hubbard', 25.95059178474007)
('Pooka Williams', 25.228339535232394)
('Kennedy Brooks', 25.060612709517322)
('Spencer Brown', 24.455426938365875)
('Elijah Mitchell', 23.797527021989673)
('Elijah Collins', 23.676010648257336)
('Michael Carter', 22.940066412976993)
('Kenny Gainwell', 21.333996481844267)
('Josh Johnson', 20.98074324890969)
('Isaiah Bowser', 20.56365113231011)
('Malik Davis', 20.37384597953665)
('JaTarvious Whitlow', 20.311547444271035)
('CJ Verdell', 20.253698804381568)
('Stevie Scott', 19.590322093696322)
('Gerrid Doaks', 19.371934920622415)
('Journey Brown', 18.427187902309054)
('Alex Fontenot', 18.311832922174414)
('Jaret Patterson', 18.23806734882422)
('Rhamondre Stevenson', 18.175768813558633)
('Jafar Armstrong', 17.57948283315939)
('B.J. Emmons', 17.327550294942498)
('Rake

In [39]:
model_1_preds_sleeper = model_1_sleeper.predict(X_sleeper_rookie)
model_1_preds_norm_sleeper = normalized_predictions_with_names(model_1_sleeper.predict(X_sleeper_rookie))

for i in model_1_preds_norm_sleeper:
  print(i)

('Travis Etienne', 43.24448276150197)
('Najee Harris', 39.48488359784922)
('Jaret Patterson', 39.190019116665916)
('Kenny Gainwell', 37.61586597496248)
('Alex Fontenot', 34.01620179516567)
('Chuba Hubbard', 31.974988497861734)
('Javonte Williams', 29.882976181503807)
('Jermar Jefferson', 29.744347884511498)
('Michael Carter', 28.502980182016355)
('Kennedy Brooks', 25.373853191092252)
('Josh Johnson', 25.278271819953005)
('CJ Verdell', 24.13968169267593)
('Pooka Williams', 23.849526098049335)
('Trey Sermon', 23.26705446239278)
('Elijah Mitchell', 22.532256812860084)
('Journey Brown', 21.948548604771574)
('Spencer Brown', 21.785275085526056)
('Brenden Knox', 21.484019859176325)
('Stevie Scott', 20.912577528054825)
('JaTarvious Whitlow', 20.89160731999107)
('Rhamondre Stevenson', 20.206943271500613)
('Jerome Ford', 18.818220722400046)
('Javian Hawkins', 17.644835372043623)
('Brittain Brown', 17.136131229087592)
('Khalil Herbert', 17.048037022594105)
('Rakeem Boyd', 16.609235354242465)
('K

In [46]:
# Create list based off differences

diff_list = list()

for i in range(len(rf_preds)):
  #diff_list.append(model_1_preds_sleeper[i]-model_1_preds[i])
  diff_list.append(rf_preds_sleeper[i]-rf_preds[i])

diff_dict = dict()

for i in range(len(diff_list)):
    if type(diff_list[i]) is np.ndarray:
      diff_dict[rookie_names[i]] = diff_list[i][0]
    else:
      diff_dict[rookie_names[i]] = diff_list[i]   
        
sorted_diff = sorted(diff_dict.items(), key=lambda x: x[1], reverse=True)

for i in sorted_diff:
  print(i)

('Trey Sermon', 2.4833999999999925)
('Javonte Williams', 2.0731500000000036)
('Jermar Jefferson', 1.913400000000001)
('Kennedy Brooks', 1.5934499999999945)
('Elijah Mitchell', 1.2378500000000017)
('Alex Fontenot', 1.106250000000002)
('Spencer Brown', 1.0378000000000007)
('Ty Chandler', 0.7240999999999982)
('Demario McCall', 0.5562500000000017)
('Deon Jackson', 0.5156500000000004)
('J.D. King', 0.510900000000003)
('Stevie Scott', 0.3849499999999999)
('JaTarvious Whitlow', 0.3824999999999976)
('Chase Hayden', 0.3562500000000006)
('Josh Johnson', 0.33325000000000227)
('Malik Davis', 0.2987500000000045)
('Kenny Gainwell', 0.28950000000000387)
('Pooka Williams', 0.28090000000000437)
('Brittain Brown', 0.27734999999999754)
('Journey Brown', 0.2610500000000018)
('Jashaun Corbin', 0.18025000000000047)
('Christopher Brown Jr.', 0.1230999999999991)
('Kene Nwangwu', 0.10385000000000044)
('Isaiah Bowser', 0.06435000000000102)
('Kylin Hill', 0.04980000000000029)
('Gerrid Doaks', 0.02954999999999685